<img src = "https://images2.imgbox.com/60/09/VFwl5LOq_o.jpg" width="400">

# 1. We'll take the CASE
---

So you've learned how to aggregate and join data from tables in your database—now what? How do you manipulate, transform, and make the most sense of your data? This intermediate-level course will teach you several key functions necessary to wrangle, filter, and categorize information in a relational database, expand your SQL toolkit, and answer complex questions. You will learn the robust use of CASE statements, subqueries, and window functions—all while discovering some interesting facts about soccer using the European Soccer Database.

In this chapter, you will learn how to use the CASE WHEN statement to create categorical variables, aggregate data into a single column with multiple filtering conditions, and calculate counts and percentages.

In [1]:
# %pip install ipython-sql

In [2]:
%load_ext sql

In [3]:
%sql sqlite:///data/soccer.db

'Connected: @data/soccer.db'

In [4]:
%%sql 

SELECT *
FROM   team
LIMIT  10 

 * sqlite:///data/soccer.db
Done.


id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
1,9987,673,KRC Genk,GEN
2,9993,675,Beerschot AC,BAC
3,10000,15005,SV Zulte-Waregem,ZUL
4,9994,2007,Sporting Lokeren,LOK
5,9984,1750,KSV Cercle Brugge,CEB
6,8635,229,RSC Anderlecht,AND
7,9991,674,KAA Gent,GEN
8,9998,1747,RAEC Mons,MON
9,7947,None,FCV Dender EH,DEN
10,9985,232,Standard de Liège,STL


## Basic CASE statements
---

What is your favorite team?

The European Soccer Database contains data about 12,800 matches from 11 countries played between 2011-2015! Throughout this course, you will be shown filtered versions of the tables in this database in order to better explore their contents.

In this exercise, you will identify matches played between FC Schalke 04 and FC Bayern Munich. There are 2 teams identified in each match in the `hometeam_id` and `awayteam_id` columns, available to you in the filtered `matches_germany` table. ID can join to the `team_api_id` column in the `teams_germany` table, but you cannot perform a join on both at the same time.

However, you can perform this operation using a `CASE` statement once you've identified the `team_api_id` associated with each team!

### Instructions

Select the team's long name and API id from the `teams_germany` table.

Filter the query for *FC Schalke 04* and *FC Bayern Munich* using `IN`, giving you the `team_api_IDs` needed for the next step.

In [5]:
%%sql 

SELECT team_long_name,
       team_api_id
FROM   teams_germany
WHERE  team_long_name IN ( 'FC Schalke 04', 'FC Bayern Munich' ) 

 * sqlite:///data/soccer.db
Done.


team_long_name,team_api_id
FC Bayern Munich,9823
FC Schalke 04,10189


Create a `CASE` statement that identifies whether a match in Germany included `FC Bayern Munich`, `FC Schalke 04`, or neither as the home team.

Group the query by the `CASE` statement alias, `home_team`.

In [6]:
%%sql 

SELECT CASE
         WHEN hometeam_id = 10189 THEN 'FC Schalke 04'
         WHEN hometeam_id = 9823 THEN 'FC Bayern Munich'
         ELSE 'Other'
       END       AS home_team,
       COUNT( id ) AS total_matches
FROM   matches_germany
GROUP  BY home_team 

 * sqlite:///data/soccer.db
Done.


home_team,total_matches
FC Bayern Munich,85
FC Schalke 04,85
Other,1360


## CASE statements comparing column values
---

Barcelona is considered one of the strongest teams in Spain's soccer league.

In this exercise, you will be creating a list of matches in the 2011/2012 season where Barcelona was the home team. You will do this using a `CASE` statement that compares the values of two columns to create a new group -- wins, losses, and ties.

In 3 steps, you will build a query that identifies a match's winner, identifies the identity of the opponent, and finally filters for Barcelona as the home team. Completing a query in this order will allow you to watch your results take shape with each new piece of information.

The `matches_spain` table currently contains Barcelona's matches from the 2011/2012 season, and has two key columns, `hometeam_id` and `awayteam_id`, that can be joined with the `teams_spain` table. However, you can only join `teams_spain` to one column at a time.

### Instructions

Select the `date` of the match and create a `CASE` statement to identify matches as home wins, home losses, or ties.

In [7]:
%%sql

SELECT date,
       CASE
         WHEN home_goal > away_goal THEN 'Home win!'
         WHEN home_goal < away_goal THEN 'Home loss :('
         ELSE 'Tie'
       END AS outcome
FROM   matches_spain
LIMIT  20 

 * sqlite:///data/soccer.db
Done.


date,outcome
2008-08-30 00:00:00,Home win!
2008-08-31 00:00:00,Tie
2008-08-31 00:00:00,Home win!
2008-08-31 00:00:00,Home win!
2008-08-31 00:00:00,Tie
2008-08-31 00:00:00,Home loss :(
2008-08-31 00:00:00,Home loss :(
2008-08-30 00:00:00,Home win!
2008-08-31 00:00:00,Home loss :(
2008-08-31 00:00:00,Home win!


Left join the `teams_spain` table `team_api_id` column to the `matches_spain` table `awayteam_id`. This allows us to retrieve the away team's identity.

Select `team_long_name` from `teams_spain` as `opponent` and complete the `CASE` statement from Step 1.

In [8]:
%%sql

SELECT m.date,
       t.team_long_name AS opponent,
       CASE
         WHEN m.home_goal > away_goal THEN 'Home win!'
         WHEN m.home_goal < away_goal THEN 'Home loss :('
         ELSE 'Tie'
       END              AS outcome
FROM   matches_spain AS m
       LEFT JOIN teams_spain AS t
              ON m.awayteam_id = t.team_api_id
LIMIT  20 

 * sqlite:///data/soccer.db
Done.


date,opponent,outcome
2008-08-30 00:00:00,RCD Mallorca,Home win!
2008-08-31 00:00:00,Villarreal CF,Tie
2008-08-31 00:00:00,Real Madrid CF,Home win!
2008-08-31 00:00:00,FC Barcelona,Home win!
2008-08-31 00:00:00,Sevilla FC,Tie
2008-08-31 00:00:00,Getafe CF,Home loss :(
2008-08-31 00:00:00,None,Home loss :(
2008-08-30 00:00:00,Real Valladolid,Home win!
2008-08-31 00:00:00,UD Almería,Home loss :(
2008-08-31 00:00:00,Málaga CF,Home win!


Complete the same `CASE` statement as the previous steps.
Filter for matches where the home team is *FC Barcelona* (id = 8634).

In [9]:
%%sql

SELECT m.date,
       t.team_long_name AS opponent,
       CASE
         WHEN m.home_goal > away_goal THEN 'Barcelona win!'
         WHEN m.home_goal < away_goal THEN 'Barcelona loss :('
         ELSE 'Tie'
       END              AS outcome
FROM   matches_spain AS m
       LEFT JOIN teams_spain AS t
              ON m.awayteam_id = t.team_api_id
WHERE  m.hometeam_id = 8634
LIMIT  20 

 * sqlite:///data/soccer.db
Done.


date,opponent,outcome
2008-11-08 00:00:00,Real Valladolid,Barcelona win!
2008-11-23 00:00:00,Getafe CF,Tie
2008-12-06 00:00:00,Valencia CF,Barcelona win!
2008-12-13 00:00:00,Real Madrid CF,Barcelona win!
2009-01-03 00:00:00,RCD Mallorca,Barcelona win!
2009-01-17 00:00:00,RC Deportivo de La Coruña,Barcelona win!
2008-09-13 00:00:00,Racing Santander,Tie
2009-01-24 00:00:00,None,Barcelona win!
2009-02-08 00:00:00,Real Sporting de Gijón,Barcelona win!
2009-02-21 00:00:00,RCD Espanyol,Barcelona loss :(


## CASE statements comparing two column values part 2
---

Similar to the previous exercise, you will construct a query to determine the outcome of Barcelona's matches where they played as the **away team**. You will learn how to combine these two queries in chapters 2 and 3.

Did their performance differ from the matches where they were the home team?

### Instructions

Complete the `CASE` statement to identify Barcelona's away team games `(id = 8634)` as wins, losses, or ties.

Left join the `teams_spain` table `team_api_id` column on the `matches_spain` table `hometeam_id` column. This retrieves the identity of the home team opponent.

Filter the query to only include matches where Barcelona was the away team.

In [10]:
%%sql

SELECT m.date,
       t.team_long_name AS opponent,
       CASE
         WHEN m.home_goal < m.away_goal THEN 'Barcelona win!'
         WHEN m.home_goal > m.away_goal THEN 'Barcelona loss :('
         ELSE 'Tie'
       END              AS outcome
FROM   matches_spain AS m
       LEFT JOIN teams_spain AS t
              ON m.hometeam_id = t.team_api_id
WHERE  m.awayteam_id = 8634
LIMIT  20 

 * sqlite:///data/soccer.db
Done.


date,opponent,outcome
2008-08-31 00:00:00,None,Barcelona loss :(
2008-11-16 00:00:00,None,Barcelona win!
2008-11-29 00:00:00,Sevilla FC,Barcelona win!
2008-12-21 00:00:00,Villarreal CF,Barcelona win!
2009-01-11 00:00:00,CA Osasuna,Barcelona win!
2009-02-01 00:00:00,Racing Santander,Barcelona win!
2009-02-14 00:00:00,Real Betis Balompié,Tie
2009-03-01 00:00:00,Atlético Madrid,Barcelona loss :(
2009-03-15 00:00:00,UD Almería,Barcelona win!
2009-04-04 00:00:00,Real Valladolid,Barcelona win!


## In CASE of rivalry
---

Barcelona and Real Madrid have been rival teams for more than 80 years. Matches between these two teams are given the name El Clásico (The Classic). In this exercise, you will query a list of matches played between these two rivals.

You will notice in Step 2 that when you have multiple logical conditions in a `CASE` statement, you may quickly end up with a large number of `WHEN` clauses to logically test every outcome you are interested in. It's important to make sure you don't accidentally exclude key information in your `ELSE` clause.

In this exercise, you will retrieve information about matches played between **Barcelona** (id = `8634`) and **Real Madrid** (id = `8633`). Note that the query you are provided with already identifies the Clásico matches using a filter in the `WHERE` clause.

### Instructions

Complete the first `CASE` statement, identifying Barcelona or Real Madrid as the home team using the `hometeam_id` column.

Complete the second `CASE` statement in the same way, using `awayteam_id`.

In [11]:
%%sql

SELECT date,
       CASE
         WHEN hometeam_id = 8634 THEN 'FC Barcelona'
         ELSE 'Real Madrid CF'
       END AS home,
       CASE
         WHEN awayteam_id = 8634 THEN 'FC Barcelona'
         ELSE 'Real Madrid CF'
       END AS away
FROM   matches_spain
WHERE  ( awayteam_id = 8634
          OR hometeam_id = 8634 )
       AND ( awayteam_id = 8633
              OR hometeam_id = 8633 ) 

 * sqlite:///data/soccer.db
Done.


date,home,away
2008-12-13 00:00:00,FC Barcelona,Real Madrid CF
2009-05-02 00:00:00,Real Madrid CF,FC Barcelona
2009-11-29 00:00:00,FC Barcelona,Real Madrid CF
2010-04-10 00:00:00,Real Madrid CF,FC Barcelona
2010-11-29 00:00:00,FC Barcelona,Real Madrid CF
2011-04-16 00:00:00,Real Madrid CF,FC Barcelona
2011-12-10 00:00:00,Real Madrid CF,FC Barcelona
2012-04-21 00:00:00,FC Barcelona,Real Madrid CF
2013-03-02 00:00:00,Real Madrid CF,FC Barcelona
2012-10-07 00:00:00,FC Barcelona,Real Madrid CF


Construct the final `CASE` statement identifying who won each match. Note there are 3 possible outcomes, but 5 conditions that you need to identify.

Fill in the logical operators to identify Barcelona or Real Madrid as the winner.

In [12]:
%%sql

SELECT date,
       CASE
         WHEN hometeam_id = 8634 THEN 'FC Barcelona'
         ELSE 'Real Madrid CF'
       END AS home,
       CASE
         WHEN awayteam_id = 8634 THEN 'FC Barcelona'
         ELSE 'Real Madrid CF'
       END AS away,
       CASE
         WHEN home_goal > away_goal
              AND hometeam_id = 8634 THEN 'Barcelona win!'
         WHEN home_goal > away_goal
              AND hometeam_id = 8633 THEN 'Real Madrid win!'
         WHEN home_goal < away_goal
              AND awayteam_id = 8634 THEN 'Barcelona win!'
         WHEN home_goal < away_goal
              AND awayteam_id = 8633 THEN 'Real Madrid win!'
         ELSE 'Tie!'
       END AS outcome
FROM   matches_spain
WHERE  ( awayteam_id = 8634
          OR hometeam_id = 8634 )
       AND ( awayteam_id = 8633
              OR hometeam_id = 8633 ) 

 * sqlite:///data/soccer.db
Done.


date,home,away,outcome
2008-12-13 00:00:00,FC Barcelona,Real Madrid CF,Barcelona win!
2009-05-02 00:00:00,Real Madrid CF,FC Barcelona,Barcelona win!
2009-11-29 00:00:00,FC Barcelona,Real Madrid CF,Barcelona win!
2010-04-10 00:00:00,Real Madrid CF,FC Barcelona,Barcelona win!
2010-11-29 00:00:00,FC Barcelona,Real Madrid CF,Barcelona win!
2011-04-16 00:00:00,Real Madrid CF,FC Barcelona,Tie!
2011-12-10 00:00:00,Real Madrid CF,FC Barcelona,Barcelona win!
2012-04-21 00:00:00,FC Barcelona,Real Madrid CF,Real Madrid win!
2013-03-02 00:00:00,Real Madrid CF,FC Barcelona,Real Madrid win!
2012-10-07 00:00:00,FC Barcelona,Real Madrid CF,Tie!


## Filtering your CASE statement
---

Let's generate a list of matches won by Italy's Bologna team! There are quite a few additional teams in the two tables, so a key part of generating a usable query will be using your `CASE` statement as a filter in the WHERE clause.

`CASE` statements allow you to categorize data that you're interested in -- and exclude data you're not interested in. In order to do this, you can use a `CASE` statement as a filter in the `WHERE` statement to remove output you don't want to see.

Here is how you might set that up:

`SELECT *
 FROM   table
 WHERE  CASE
          WHEN a > 5 THEN 'Keep'
          WHEN a <= 5 THEN 'Exclude'
        END = 'Keep' `
         
In essence, you can use the `CASE` statement as a filtering column like any other column in your database. The only difference is that you don't alias the statement in `WHERE`.

### Instructions

Identify Bologna's team ID listed in the `teams_italy` table by selecting the `team_long_name` and `team_api_id`.

In [13]:
%%sql

SELECT team_long_name,
       team_api_id
FROM   teams_italy
WHERE  team_long_name = 'Bologna' 

 * sqlite:///data/soccer.db
Done.


team_long_name,team_api_id
Bologna,9857


Select the `season` and `date` that a match was played.

Complete the `CASE` statement so that only Bologna's home and away wins are identified.

In [14]:
%%sql

SELECT season,
       date,
       CASE
         WHEN hometeam_id = 9857
              AND home_goal > away_goal THEN 'Bologna Win'
         WHEN awayteam_id = 9857
              AND away_goal > home_goal THEN 'Bologna Win'
       END AS outcome
FROM   matches_italy
LIMIT  20

 * sqlite:///data/soccer.db
Done.


season,date,outcome
2008/2009,2008-08-31 00:00:00,None
2008/2009,2008-08-31 00:00:00,None
2008/2009,2008-08-31 00:00:00,None
2008/2009,2008-08-31 00:00:00,None
2008/2009,2008-08-31 00:00:00,None
2008/2009,2008-08-31 00:00:00,Bologna Win
2008/2009,2008-08-31 00:00:00,None
2008/2009,2008-08-30 00:00:00,None
2008/2009,2008-08-31 00:00:00,None
2008/2009,2008-08-30 00:00:00,None


Select the `home_goal` and `away_goal` for each match.

Use the `CASE` statement in the `WHERE` clause to filter all `NULL` values generated by the statement in the previous step.

In [15]:
%%sql

SELECT season,
       date,
       home_goal,
       away_goal
FROM   matches_italy
WHERE  CASE
         WHEN hometeam_id = 9857
              AND home_goal > away_goal THEN 'Bologna Win'
         WHEN awayteam_id = 9857
              AND away_goal > home_goal THEN 'Bologna Win'
       END IS NOT NULL
LIMIT  20 

 * sqlite:///data/soccer.db
Done.


season,date,home_goal,away_goal
2008/2009,2008-08-31 00:00:00,1,2
2008/2009,2008-12-13 00:00:00,5,2
2008/2009,2009-01-18 00:00:00,1,2
2008/2009,2009-01-28 00:00:00,0,1
2008/2009,2009-03-08 00:00:00,3,0
2008/2009,2009-04-26 00:00:00,2,0
2008/2009,2009-05-17 00:00:00,2,1
2008/2009,2009-05-31 00:00:00,3,1
2008/2009,2008-10-19 00:00:00,3,1
2009/2010,2009-10-28 00:00:00,2,1


## COUNT using CASE WHEN
---

Do the number of soccer matches played in a given European country differ across seasons? We will use the European Soccer Database to answer this question.

You will examine the number of matches played in 3 seasons within each country listed in the database. This is much easier to explore with each season's matches in separate columns. Using the `country` and unfiltered `match` table, you will count the number of matches played in each country during the 2012/2013, 2013/2014, and 2014/2015 match seasons.

### Instructions

Create a `CASE` statement that identifies the `id` of matches played in the 2012/2013 season. Specify that you want `ELSE` values to be `NULL`.

Wrap the `CASE` statement in a `COUNT` function and group the query by the `country`
alias.

In [16]:
%%sql

SELECT c.name     AS country,
       Count(CASE
               WHEN m.season = '2012/2013' THEN m.id
               ELSE NULL
             END) AS matches_2012_2013
FROM   country AS c
       LEFT JOIN match AS m
              ON c.id = m.country_id
GROUP  BY country 

 * sqlite:///data/soccer.db
Done.


country,matches_2012_2013
Belgium,240
England,380
France,380
Germany,306
Italy,380
Netherlands,306
Poland,240
Portugal,240
Scotland,228
Spain,380


Create 3 `CASE WHEN` statements counting the matches played in each country across the 3 seasons.

`END` your `CASE` statement without an `ELSE` clause.

In [17]:
%%sql

SELECT c.name     AS country,
       Count(CASE
               WHEN m.season = '2012/2013' THEN m.id
             END) AS matches_2012_2013,
       Count(CASE
               WHEN m.season = '2013/2014' THEN m.id
             END) AS matches_2013_2014,
       Count(CASE
               WHEN m.season = '2014/2015' THEN m.id
             END) AS matches_2014_2015
FROM   country AS c
       LEFT JOIN match AS m
              ON c.id = m.country_id
GROUP  BY country 

 * sqlite:///data/soccer.db
Done.


country,matches_2012_2013,matches_2013_2014,matches_2014_2015
Belgium,240,12,240
England,380,380,380
France,380,380,380
Germany,306,306,306
Italy,380,380,379
Netherlands,306,306,306
Poland,240,240,240
Portugal,240,240,306
Scotland,228,228,228
Spain,380,380,380


## COUNT and CASE WHEN with multiple conditions
---

In R or Python, you have the ability to calculate a `SUM` of logical values (i.e., `TRUE/FALSE`) directly. In SQL, you have to convert these values into 1 and 0 before calculating a sum. This can be done using a `CASE` statement.

There's one key difference when using `SUM` to aggregate logical values compared to using `COUNT` in the previous exercise --

Your goal here is to use the `country` and `match` table to determine the total number of matches won by the home team in each country during the 2012/2013, 2013/2014, and 2014/2015 seasons.

### Instructions

Create 3 `CASE` statements to "count" matches in the `'2012/2013'`, `'2013/2014'`, and `'2014/2015'` seasons, respectively.

Have each `CASE` statement return a `1` for every match you want to include, and a `0` for every match to exclude.

Wrap the `CASE` statement in a `SUM` to return the total matches played in each season.

Group the query by the country name alias.

In [18]:
%%sql

SELECT c.name   AS country,
       SUM(CASE
             WHEN m.season = '2012/2013'
                  AND m.home_goal > m.away_goal THEN 1
             ELSE 0
           END) AS matches_2012_2013,
       SUM(CASE
             WHEN m.season = '2013/2014'
                  AND m.home_goal > m.away_goal THEN 1
             ELSE 0
           END) AS matches_2013_2014,
       SUM(CASE
             WHEN m.season = '2014/2015'
                  AND m.home_goal > m.away_goal THEN 1
             ELSE 0
           END) AS matches_2014_2015
FROM   country AS c
       LEFT JOIN match AS m
              ON c.id = m.country_id
GROUP  BY country 

 * sqlite:///data/soccer.db
Done.


country,matches_2012_2013,matches_2013_2014,matches_2014_2015
Belgium,102,6,106
England,166,179,172
France,170,168,181
Germany,130,145,145
Italy,177,181,152
Netherlands,137,144,138
Poland,97,110,114
Portugal,103,108,137
Scotland,89,102,102
Spain,189,179,171


## Calculating percent with CASE and AVG
---

`CASE` statements will return any value you specify in your `THEN` clause. This is an incredibly powerful tool for robust calculations and data manipulation when used in conjunction with an aggregate statement. One key task you can perform is using `CASE` inside an `AVG` function to calculate a percentage of information in your database.

Here's an example of how you set that up:

`AVG(CASE WHEN condition_is_met THEN 1
         WHEN condition_is_not_met THEN 0 END)`
         
With this approach, it's important to accurately specify which records count as `0`, otherwise your calculations may not be correct!

Your task is to examine the number of wins, losses, and ties in each country. The `matches` table is filtered to include all matches from the 2013/2014 and 2014/2015 seasons.

### Instructions

Create 3 `CASE` statements to `COUNT` the total number of home team wins, away team wins, and ties, which will allow you to examine the total number of records.

In [19]:
%%sql

SELECT c.name     AS country,
       COUNT(CASE
               WHEN m.home_goal > m.away_goal THEN m.id
             END) AS home_wins,
       COUNT(CASE
               WHEN m.home_goal < m.away_goal THEN m.id
             END) AS away_wins,
       COUNT(CASE
               WHEN m.home_goal = m.away_goal THEN m.id
             END) AS ties
FROM   country AS c
       LEFT JOIN match AS m
              ON c.id = m.country_id
GROUP  BY country 

 * sqlite:///data/soccer.db
Done.


country,home_wins,away_wins,ties
Belgium,214,157,121
England,688,460,372
France,698,410,412
Germany,559,362,303
Italy,675,415,407
Netherlands,574,352,298
Poland,428,265,267
Portugal,463,304,259
Scotland,383,305,224
Spain,727,438,355


Calculate the percentage of matches tied using a `CASE` statement inside `AVG`.

Fill in the logical operators for each statement. Alias your columns as `ties_2013_2014` and `ties_2014_2015`, respectively.

In [20]:
%%sql

SELECT c.name   AS country,
       AVG(CASE
             WHEN m.season = '2013/2014'
                  AND m.home_goal = m.away_goal THEN 1
             WHEN m.season = '2013/2014'
                  AND m.home_goal != m.away_goal THEN 0
           END) AS ties_2013_2014,
       AVG(CASE
             WHEN m.season = '2014/2015'
                  AND m.home_goal = m.away_goal THEN 1
             WHEN m.season = '2014/2015'
                  AND m.home_goal != m.away_goal THEN 0
           END) AS ties_2014_2015
FROM   country AS c
       LEFT JOIN match AS m
              ON c.id = m.country_id
GROUP  BY country 

 * sqlite:///data/soccer.db
Done.


country,ties_2013_2014,ties_2014_2015
Belgium,0.16666666666666666,0.25
England,0.20526315789473684,0.24473684210526317
France,0.28421052631578947,0.23157894736842105
Germany,0.20915032679738563,0.2679738562091503
Italy,0.23684210526315788,0.316622691292876
Netherlands,0.27450980392156865,0.238562091503268
Poland,0.30416666666666664,0.275
Portugal,0.25,0.2777777777777778
Scotland,0.21929824561403508,0.19298245614035087
Spain,0.22631578947368422,0.2394736842105263


The previous "ties" columns returned values with 14 decimal points, which is not easy to interpret. 

Use the `ROUND` function to round to 2 decimal points.

In [21]:
%%sql

SELECT c.NAME             AS country,
       ROUND(AVG(CASE
                   WHEN m.season = '2013/2014'
                        AND m.home_goal = m.away_goal THEN 1
                   WHEN m.season = '2013/2014'
                        AND m.home_goal != m.away_goal THEN 0
                 END), 2) AS pct_ties_2013_2014,
       ROUND(AVG(CASE
                   WHEN m.season = '2014/2015'
                        AND m.home_goal = m.away_goal THEN 1
                   WHEN m.season = '2014/2015'
                        AND m.home_goal != m.away_goal THEN 0
                 END), 2) AS pct_ties_2014_2015
FROM   country AS c
       LEFT JOIN match AS m
              ON c.id = m.country_id
GROUP  BY country 

 * sqlite:///data/soccer.db
Done.


country,pct_ties_2013_2014,pct_ties_2014_2015
Belgium,0.17,0.25
England,0.21,0.24
France,0.28,0.23
Germany,0.21,0.27
Italy,0.24,0.32
Netherlands,0.27,0.24
Poland,0.3,0.28
Portugal,0.25,0.28
Scotland,0.22,0.19
Spain,0.23,0.24
